# Geração de Visualizações para o Projeto de Segmentação de Clientes

Este notebook é dedicado à geração das visualizações para o README do projeto.


In [2]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurar o estilo das visualizações
sns.set_theme(style="whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

# Criar diretório para imagens se não existir
if not os.path.exists('images'):
    os.makedirs('images')


In [3]:
# Carregar os dados
try:
    df = pd.read_csv('Mall_Customers.csv')
    print("Dataset carregado com sucesso!")
    print("\nInformações do dataset:")
    print(df.info())
    print("\nPrimeiras linhas do dataset:")
    print(df.head())
except FileNotFoundError:
    print("Erro: Arquivo 'Mall_Customers.csv' não encontrado!")
    print("Por favor, verifique se o arquivo está no diretório correto.")
    raise

# 1. Análise Exploratória
plt.figure(figsize=(15, 10))

# Distribuições
plt.subplot(2, 2, 1)
sns.histplot(data=df, x='Age', kde=True, color='skyblue')
plt.title('Distribuição de Idade')

plt.subplot(2, 2, 2)
sns.histplot(data=df, x='Annual Income (k$)', kde=True, color='lightgreen')
plt.title('Distribuição de Renda Anual')

plt.subplot(2, 2, 3)
sns.histplot(data=df, x='Spending Score (1-100)', kde=True, color='salmon')
plt.title('Distribuição de Score de Gastos')

plt.subplot(2, 2, 4)
sns.countplot(data=df, x='Gender', palette=['lightblue', 'lightpink'])
plt.title('Distribuição de Gênero')

plt.suptitle('Análise Exploratória dos Dados', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig('images/exploratory_analysis.png', dpi=300, bbox_inches='tight')
plt.close()


Dataset carregado com sucesso!

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
None

Primeiras linhas do dataset:
   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                      39
1           2    Male   21                  15                      81
2           3  Female   20                  16                       6
3           4  Female   23                  16                      77
4           5  Female   31          

/var/folders/pr/wx16k7494f1grpgyylpct75h0000gn/T/ipykernel_2443/615261412.py:31: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x='Gender', palette=['lightblue', 'lightpink'])


In [4]:
# 2. Matriz de Correlação
plt.figure(figsize=(10, 8))
numeric_cols = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.savefig('images/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.close()


In [5]:
# 3. Preparar dados para K-means
scaler = StandardScaler()
X = scaler.fit_transform(df[numeric_cols])

# Calcular inércia e silhouette score para diferentes valores de k
k_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))

# Plotar Método do Cotovelo e Silhouette Score
plt.figure(figsize=(15, 5))

# Método do Cotovelo
plt.subplot(1, 2, 1)
plt.plot(k_range, inertias, 'bx-')
plt.xlabel('k (número de clusters)')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo')
plt.grid(True)

# Silhouette Score
plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, 'rx-')
plt.xlabel('k (número de clusters)')
plt.ylabel('Silhouette Score')
plt.title('Análise do Silhouette Score')
plt.grid(True)

plt.tight_layout()
plt.savefig('images/metodos_avaliacao.png', dpi=300, bbox_inches='tight')
plt.close()


In [6]:
# 4. Visualização dos Clusters
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
k_values = [3, 4, 5]

for idx, k in enumerate(k_values):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X[:, :2])
    
    scatter = axes[idx].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
    axes[idx].set_xlabel('Primeira Característica')
    axes[idx].set_ylabel('Segunda Característica')
    axes[idx].set_title(f'Visualização dos Clusters (k={k})')
    plt.colorbar(scatter, ax=axes[idx])

plt.tight_layout()
plt.savefig('images/clusters_visualization.png', dpi=300, bbox_inches='tight')
plt.close()
